In [ ]:
import sys

import autograd.numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

sys.path.append("..")

import shared.format
import shared.tools

In [ ]:
plt.rcParams["font.size"] = shared.format.FONTSIZE

In [ ]:
def babylon_step(theta_t, x):
    return theta_t / 2  + x / theta_t / 2

In [ ]:
def babylon_iterate(T, x, theta_0):
    thetas = np.zeros(T + 1)
    thetas[0] = theta_0
    for tt in range(T):
        thetas[tt + 1] = babylon_step(thetas[tt], x)
    return thetas

In [ ]:
def make_square(side_length, color, **kwargs):
    color_as_array = np.array(mpl.colors.to_rgb(color))
    face_color = 0.5 * color_as_array + 0.5 * 0.67 * np.ones(3)
    return mpl.patches.Rectangle(
        (0, 1), side_length, -side_length,
        fill=True,
        facecolor=face_color, edgecolor=color, lw=shared.format.LINEWIDTH,
        **kwargs)

def make_babylon_diagram(x, theta, square_colors=["C1", "xkcd:brick", "k", "C0"], ax=None,
                         axis_offset=0.25, text_offset=0.1):
    if ax is None:
        f, ax = plt.subplots(figsize=(8, 8))
    x_div_theta = x / theta
    next_theta = babylon_step(theta, x)
    square_lengths = [theta, next_theta, np.sqrt(x), x_div_theta]
    squares = [make_square(length, color=color)
               for length, color in zip(square_lengths, square_colors)]
    [ax.add_artist(square) for square in squares]
    ax.axis("off")
    ax.axis("equal")
    ax.set_xlim(-axis_offset, max(square_lengths) + axis_offset)
    ax.set_ylim(1 - max(square_lengths) - axis_offset, 1 + axis_offset)
    
    label_size = shared.format.FONTSIZE + 6
    xpos_verts = -text_offset
    ypos_horzs = 1 + text_offset
    vert_kwargs = {"fontsize": label_size,
                   "rotation": "vertical",
                   "horizontalalignment": "center",
                   "verticalalignment": "bottom"}
    horz_kwargs = {"fontsize": label_size,
                   "rotation": "horizontal",
                   "horizontalalignment": "left",
                   "verticalalignment": "center"}

    ax.text(xpos_verts, 1 - np.sqrt(x), r"$\sqrt{x}$",
            color=square_colors[2], **vert_kwargs)
    ax.text(xpos_verts, 1 - x_div_theta, r"$x \div \theta_t$",
            color=square_colors[3], **vert_kwargs)
    
    ax.text(next_theta, ypos_horzs, r"$\theta_{t+1}$",
            color=square_colors[1], **horz_kwargs)
    ax.text(theta, ypos_horzs, r"$\theta_{t}$",
            color=square_colors[0], **horz_kwargs)

In [ ]:
fig = plt.figure(figsize=(12, 6))
grid = mpl.gridspec.GridSpec(2, 2, figure=fig)

T = 6
x = 4
theta_0 = 0.8
iterates = babylon_iterate(T, x, theta_0)

iteration_to_plot = 1
theta_t = iterates[1]

diagram_ax = fig.add_subplot(grid[:, 0])
make_babylon_diagram(x, theta_t, ax=diagram_ax, axis_offset=0.2, text_offset=0.11)

value_ax = fig.add_subplot(grid[0, 1])
value_ax.plot(iterates, lw=shared.format.LINEWIDTH)
xlims = value_ax.get_xlim()
value_ax.hlines(np.sqrt(x), xlims[0] - 1, xlims[1] + 1,
                zorder=1, lw=shared.format.LINEWIDTH - 2, color="k", linestyle="--")
value_ax.set_xlim(*xlims)
value_ax.set_ylabel(r"$\theta$", size=shared.format.FONTSIZE + 2)
value_ax.scatter(iteration_to_plot, theta_t, color="C1", zorder=3, s=81)
value_ax.scatter(iteration_to_plot + 1, iterates[iteration_to_plot + 1],
                 color="xkcd:brick", zorder=3, s=81)

error_ax = fig.add_subplot(grid[1, 1], sharex=value_ax)
error_ax.semilogy(np.abs(iterates - 2), lw=shared.format.LINEWIDTH);
error_ax.set_ylabel("Absolute Error", size=shared.format.FONTSIZE + 2);
error_ax.set_xlabel("Iteration $t$", size=shared.format.FONTSIZE + 2);
error_ax.set_yticks([1e0, 1e-5, 1e-10])

plt.tight_layout()
shared.tools.add_panel_label("A", diagram_ax, pos=(0, 1.06))
shared.tools.add_panel_label("B", value_ax)
shared.tools.add_panel_label("C", error_ax)
plt.savefig("babylon.pdf")